In [1]:
# imports
import time

import psycopg2
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import pandas as pd
from IPython.display import display, clear_output

In [2]:
# open/restart connection to chaind database
try:
    cursor.close()
    connection.close()
except:
    pass

connection = psycopg2.connect(user="chain", host="127.0.0.1", database="chain", password="medalla")
cursor = connection.cursor()

In [3]:
# get info about dataset and validators

cursor.execute("SELECT MAX(f_slot) FROM t_blocks")
latest_slot = cursor.fetchone()[0]
n_slots = latest_slot + 1
n_epochs = (n_slots - (n_slots % 32)) // 32

cursor.execute("SELECT f_activation_eligibility_epoch, f_activation_epoch, f_exit_epoch, f_slashed, "
               "f_first_attested_epoch, f_latest_attested_epoch FROM t_validators ORDER BY f_index")
result = cursor.fetchall()
validators = [{"activation_eligibility_epoch": r[0],
               "activation_epoch"            : r[1],
               "exit_epoch"                  : r[2],
               "slashed"                     : r[3],
               "first_attestation"           : r[4],
               "latest_attestation"          : r[5]} for r in result]

In [4]:
print("slot total correct incorrect missed")
for slot in range(10):
    cursor.execute(f"SELECT f_correct FROM t_validator_performance WHERE f_slot = {slot}")
    correct = cursor.fetchone()[0]
    n_total     = len(correct)
    n_correct   = sum([c ==  1 for c in correct])
    n_incorrect = sum([c == -1 for c in correct])
    n_missed    = sum([c ==  0 for c in correct])
    print(f"{slot}".rjust(3,' '), f"{n_total}".rjust(5,' '), f"{n_correct}".rjust(6,' '),
          f"{n_incorrect}".rjust(7,' '), f"{n_missed}".rjust(8,' '))

slot total correct incorrect missed
  0   627    332       0      295
  1   628    339      54      235
  2   627    394      13      220
  3   628    347      57      224
  4   628    348      46      234
  5   627    342      33      252
  6   628    342      44      242
  7   628    337      52      239
  8   627    341      46      240
  9   628    327      62      239


In [16]:
cursor.execute(f"SELECT f_slot, f_root FROM t_blocks WHERE f_slot <= {att_slot} AND f_canonical 
               f"ORDER BY f_slot DESC LIMIT 1")
result = cursor.fetchone()
slot, root = result[0], result[1].hex()
print(slot, root)

1 4f042e61cbf74a5064584f761951679c92844d8b314c97a37a516fba651cda16


In [17]:
cursor.execute(f"SELECT SUM(f_canonical::int) FROM t_blocks WHERE f_slot < 10")
cursor.fetchone()

(9,)